# Ответы
### 1) secondary
### 2) pdays и previous
### 3) poutcome
### 4) 0.9
### 5) previous
### 6) 1

In [2]:
import pandas as pd

df = pd.read_csv('bank-full.csv', sep=';')

columns = ['age', 'job', 'marital', 'education', 'balance', 'housing', 'contact',
           'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
df = df[columns]

missing_values = df.isnull().sum()
print("Пропущенные значения:\n", missing_values)

Пропущенные значения:
 age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64


### 1 вопрос

In [3]:
education_mode = df['education'].mode()[0]

print("Самое частое значение для столбца 'education':", education_mode)

Самое частое значение для столбца 'education': secondary


### 2 вопрос

In [4]:
numeric_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
correlation_matrix = df[numeric_columns].corr()

# Поиск пары признаков с наибольшей корреляцией
max_corr_pair = correlation_matrix.unstack().sort_values(ascending=False)
max_corr_pair = max_corr_pair[max_corr_pair < 1].idxmax()

print("Пара признаков с наибольшей корреляцией:", max_corr_pair)

Пара признаков с наибольшей корреляцией: ('pdays', 'previous')


### продолжение

In [5]:
from sklearn.model_selection import train_test_split
df['y'] = df['y'].map({'yes': 1, 'no': 0})

# Разделение данных на тренировочные, валидационные и тестовые наборы
X = df.drop('y', axis=1)
y = df['y']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Размеры выборок: тренировочная:", len(X_train), ", валидационная:", len(X_val), ", тестовая:", len(X_test))

Размеры выборок: тренировочная: 27126 , валидационная: 9042 , тестовая: 9043


In [15]:
print(df.dtypes) # Что там у нас категориального такого

age           int64
job          object
marital      object
education    object
balance       int64
housing      object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y             int64
dtype: object


### 3 вопрос

In [16]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

categorical_columns = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

# Оставляем только категориальные признаки в тренировочной выборке
X_train_categorical = X_train[categorical_columns]
X_train_encoded = pd.get_dummies(X_train_categorical, drop_first=True)

# Рассчитываем взаимную информацию
mutual_info = mutual_info_classif(X_train_encoded, y_train, discrete_features=True, random_state=42)

mi_df = pd.DataFrame({'Feature': X_train_encoded.columns, 'Mutual Information': mutual_info})

# Округляем результаты до двух знаков после запятой
mi_df['Mutual Information'] = mi_df['Mutual Information'].round(2)

# Сортируем по взаимной информации
mi_df_sorted = mi_df.sort_values(by='Mutual Information', ascending=False)

print("Категориальная переменная с наибольшей взаимной информацией:", mi_df_sorted.iloc[0]['Feature'])
print("Взаимная информация для категориальных переменных:\n", mi_df_sorted)

Категориальная переменная с наибольшей взаимной информацией: poutcome_success
Взаимная информация для категориальных переменных:
                 Feature  Mutual Information
31     poutcome_success                0.03
32     poutcome_unknown                0.01
16          housing_yes                0.01
18      contact_unknown                0.01
26            month_may                0.01
28            month_oct                0.01
0       job_blue-collar                0.00
3        job_management                0.00
1      job_entrepreneur                0.00
8        job_technician                0.00
9        job_unemployed                0.00
11      marital_married                0.00
2         job_housemaid                0.00
5     job_self-employed                0.00
6          job_services                0.00
7           job_student                0.00
4           job_retired                0.00
15    education_unknown                0.00
14   education_tertiary           

### 4 вопрос

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_encoded = pd.get_dummies(X, drop_first=True)

# Создание модели логистической регрессии
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

# Обучение модели на тренировочных данных с one-hot кодированием
model.fit(X_encoded.loc[X_train.index], y_train)
# Применяем one-hot кодирование к валидационному набору данных
X_val_encoded = pd.get_dummies(X_val)
# Убедимся, что все столбцы из тренировочного набора присутствуют в валидационном
X_val_encoded = X_val_encoded.reindex(columns=X_encoded.columns, fill_value=0)
# Предсказание на валидационной выборке
y_val_pred = model.predict(X_val_encoded)

# Рассчет точности
accuracy = accuracy_score(y_val, y_val_pred)
# Округление
accuracy_rounded = round(accuracy, 2)

print("Точность на валидационном наборе данных:", accuracy_rounded)

Точность на валидационном наборе данных: 0.9


### 5 вопрос

In [29]:
# Исходная модель с использованием всех признаков
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_encoded.loc[X_train.index], y_train)

# Предсказание и расчет начальной точности
y_train_pred = model.predict(X_encoded.loc[X_train.index])
initial_accuracy = accuracy_score(y_train, y_train_pred)
print("Начальная точность:", round(initial_accuracy, 2))

# Признаки для исключения (проверяем, что они есть в X_encoded)
features_to_exclude = ['age', 'balance', 'marital', 'previous']
valid_features_to_exclude = [feature for feature in features_to_exclude if feature in X_encoded.columns]
results = {}

for feature in valid_features_to_exclude:
    # Копируем X_encoded без одного признака
    X_temp = X_encoded.drop(columns=[feature], errors='ignore')
    
    # Обучаем модель без этого признака
    model.fit(X_temp.loc[X_train.index], y_train)
    
    # Предсказание и расчет точности
    y_temp_pred = model.predict(X_temp.loc[X_train.index])
    accuracy_without_feature = accuracy_score(y_train, y_temp_pred)
    
    # Сохраняем разницу в точности
    difference = initial_accuracy - accuracy_without_feature
    results[feature] = difference

for feature, difference in results.items():
    print(f"Разница в точности без {feature}: {round(difference, 5)}")

least_useful_feature = min(results, key=results.get)
print("Наименее полезный признак:", least_useful_feature)

Начальная точность: 0.9
Разница в точности без age: -0.00037
Разница в точности без balance: -0.00026
Разница в точности без previous: -0.00041
Наименее полезный признак: previous


### 6 вопрос

In [36]:
C_values = [0.01, 0.1, 1, 10, 100]
accuracies = {}

for C in C_values:
    # Создание модели логистической регрессии с текущим значением C
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_encoded.loc[X_train.index], y_train)
    X_val_encoded = pd.get_dummies(X_val)
    X_val_encoded = X_val_encoded.reindex(columns=X_encoded.columns, fill_value=0)
    y_val_pred = model.predict(X_val_encoded)

    accuracy = accuracy_score(y_val, y_val_pred)
    accuracies[C] = round(accuracy, 3)

for C, acc in accuracies.items():
    print(f"C: {C}, Точность: {acc}")

# Находим значение C с наилучшей точностью
best_C = min(accuracies, key=lambda k: (-accuracies[k], k))
print("Наилучшее значение C:", best_C, "с точностью:", accuracies[best_C])

C: 0.01, Точность: 0.898
C: 0.1, Точность: 0.9
C: 1, Точность: 0.901
C: 10, Точность: 0.9
C: 100, Точность: 0.901
Наилучшее значение C: 1 с точностью: 0.901
